In [ ]:
# Imports
import os
from dotenv import load_dotenv
from typing import Dict, Any, List
import ipywidgets as widgets
from IPython.display import display, clear_output

from langchain_anthropic import ChatAnthropic
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field

from langgraph.graph import Graph
from langgraph.prebuilt import ToolNode

# Load environment variables
load_dotenv()

# Initialize LLM
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("ANTHROPIC_API_KEY environment variable is not set")

llm = ChatAnthropic(model="claude-3-opus-20240229",
                    anthropic_api_key=anthropic_api_key)


: 

In [14]:

# Define ScheduleTool
class ScheduleInput(BaseModel):
    action: str = Field(...,
                        description="The action to perform on the schedule (add, remove, get)")
    time: str = Field(..., description="The time for the task in HH:MM format")
    task: str = Field(..., description="The task description")


class ScheduleTool(BaseTool):
    name = "schedule_tool"
    description = "Use this tool to manage the daily schedule. Actions: add, remove, get"
    args_schema = ScheduleInput
    schedule: Dict[str, str] = {}

    def _run(self, action: str, time: str, task: str) -> Dict[str, Any]:
        if action == "add":
            self.schedule[time] = task
            return {"status": "success", "message": f"Added task '{task}' at {time}"}
        elif action == "remove":
            if time in self.schedule:
                del self.schedule[time]
                return {"status": "success", "message": f"Removed task at {time}"}
            else:
                return {"status": "error", "message": f"No task found at {time}"}
        elif action == "get":
            return {"status": "success", "schedule": self.schedule}
        else:
            return {"status": "error", "message": f"Invalid action: {action}"}

    async def _arun(self, action: str, time: str, task: str) -> Dict[str, Any]:
        return self._run(action, time, task)


# Initialize ScheduleTool and ToolNode
schedule_tool = ScheduleTool()
tool_node = ToolNode(tools=[schedule_tool])



In [17]:
# Define graph nodes


def generate_response(state):
    messages = state["messages"]
    tool_results = state.get("tool_results", [])

    if tool_results:
        messages.append(AIMessage(
            content=f"I've updated the schedule. Here's what I did:\n{tool_results[-1]}"))

    response = llm.invoke(messages)
    return {"messages": messages + [response], "response": response.content}

def call_tool(state):
    messages = state["messages"]
    response = state["response"]

    tool_input = {
        "name": "schedule_tool",
        "arguments": {
            "action": "get",
            "time": "00:00",
            "task": ""
        }
    }

    return {"tool_input": tool_input}


def should_call_tool(state):
    response = state["response"]
    if "schedule" in response.lower():
        return "call_tool"
    else:
        return "end"



In [16]:

# Create the graph
workflow = Graph()

workflow.add_node("generate_response", generate_response)
workflow.add_node("call_tool", call_tool)
workflow.add_node("tool", tool_node)

workflow.set_entry_point("generate_response")
workflow.add_edge("generate_response", should_call_tool)
workflow.add_edge("call_tool", "tool")
workflow.add_edge("tool", "generate_response")

graph = workflow.compile()

# Function to run the chat


def chat(user_input: str, messages: List):
    messages.append(HumanMessage(content=user_input))
    result = graph.invoke({"messages": messages})
    return result["messages"], result["response"]


# Initialize the chat
system_message = SystemMessage(content="""You are an AI assistant named HAL-9001, designed to help users manage their daily schedule. Your personality is slightly sarcastic and witty, but always helpful. You can add tasks to the schedule, remove them, and retrieve the current schedule. Use the schedule_tool when needed to manage the schedule. Always confirm changes with the user before making them.""")

messages = [system_message]


ValueError: Node `call_tool` is not reachable

In [ ]:

# Create widgets for the chat interface
output = widgets.Output()
text_input = widgets.Text(
    description="You:", placeholder="Type your message here...")
send_button = widgets.Button(description="Send")


def on_send_button_clicked(b):
    user_input = text_input.value
    text_input.value = ""

    with output:
        print(f"You: {user_input}")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("HAL-9001: Farewell, human. Try not to miss me too much.")
            print("\nYour final schedule:")
            final_schedule = schedule_tool._run("get", "", "")["schedule"]
            for time, task in sorted(final_schedule.items()):
                print(f"{time}: {task}")
        else:
            messages, response = chat(user_input, messages)
            print(f"HAL-9001: {response}")


send_button.on_click(on_send_button_clicked)

# Display the chat interface
display(widgets.VBox([output, widgets.HBox([text_input, send_button])]))

# Initial greeting
with output:
    print("HAL-9001: Greetings, human. I'm HAL-9001, your slightly sarcastic but incredibly efficient scheduling assistant. How may I optimize your day?")